In [1]:
import ee,geemap
import os

Map = geemap.Map()

path = r'C:\Users\xiong\Documents\geemap\shp\jxwgs\jx.shp'

ee_shp = geemap.shp_to_ee(path)
jx = ee_shp.geometry()

# Map.addLayer(ee_shp,{},'jx')

In [2]:
LUCC = ee.ImageCollection("ESA/WorldCover/v100").mosaic()
# '''在样本区域内生成随机点。'''
points = ee.FeatureCollection.randomPoints(region= jx,points=2000,seed= 1234)
# '''提取值到点'''
# print(points.getInfo())
sample = LUCC.sampleRegions(collection = points,scale =  5 ,geometries = True)
# '''提取水稻的点'''
# sample = samples.filter(ee.Filter.eq('Map',40))

# Map.addLayer(sample,{},'sample')

print(sample.size().getInfo())

2000


In [3]:
days=['31','28','31','30','31','30','31','31','30','31','30','31']
l8bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
l7bands = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7']
l5bands = ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7']
s2bands = ['B2','B3','B4','B8','B11','B12']
new_bands = ['B1','B2','B3','B4','B5','B6']
# new_bands_ = ['B1','B2','B3','B4','B5','B6','NDVI','EVI','LSWI','NDWI','GCVI']
new_bands_ = ['B1','B2','B3','B4','B5','B6','NDVI','LSWI','N_L']
             
# 去云函数
# Function to cloud mask from the pixel_qa band of Landsat 8 SR data.
def maskL8sr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Cirrus
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(31).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
      .addBands(thermalBands, None, True) \
      .updateMask(qaMask) \
      .updateMask(saturationMask).copyProperties(image, ["system:index","system:time_start"])

def maskL457sr(image):
    # Bit 0 - Fill
    # Bit 1 - Dilated Cloud
    # Bit 2 - Unused
    # Bit 3 - Cloud
    # Bit 4 - Cloud Shadow
    qaMask = image.select('QA_PIXEL').bitwiseAnd(31).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
      .addBands(thermalBand, None, True) \
      .updateMask(qaMask) \
      .updateMask(saturationMask).copyProperties(image, ["system:index","system:time_start"])


def cloudfunction_ST2(image):
    # use add the cloud likelihood band to the image
    qa = image.select("QA60")
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000)\
        .select("B.*")\
        .copyProperties(image, ["system:index","system:time_start"])


    
def NDVI(img):
    ndvi = img.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return ndvi #return img.addBands(ndvi).updateMask(mask).clipToCollection(shp)


def EVI(img):
    nir = img.select("B4")
    red = img.select("B3")
    blue = img.select("B1")
    evi = img.expression("2.5*(B4 - B3)/(B4 + 6*B3 - 7.5*B1 + 1)",{"B4": nir,"B3": red ,"B1": blue})
    evi = evi.rename('EVI')
    return evi

def LSWI(img):
    lswi = img.normalizedDifference(['B4', 'B5']).rename('LSWI')
    lswi = lswi.rename('LSWI')
    return lswi

def NDWI(img):
    ndwi = img.normalizedDifference(['B2', 'B4']).rename('NDWI')
    return ndwi

def GCVI(img):
    nir = img.select("B4")
    green = img.select("B2")
    gcvi = img.expression("B4/B2-1",{"B4": nir,"B2": green})
    gcvi = gcvi.rename('GCVI')
    return gcvi

def RVI(img):
    nir = img.select("B4")
    red = img.select("B3")
    rvi = img.expression("B4/B3",{"B4": nir,"B3": red})
    rvi = rvi.rename('RVI')
    return rvi

def EVI2(img):
    nir = img.select("B4")
    red = img.select("B3")
    evi2 = img.expression("2.5*(B4 - B3)/(B4 + 2.4*B3 + 1)",{"B4": nir,"B3": red})
    evi2 = evi2.rename('EVI2')
    return evi2

def SAVI(img):
    nir = img.select("B4")
    red = img.select("B3")
    savi = img.expression("1.5*(B4 - B3)/(B4 + B3 + 0.5)",{"B4": nir,"B3": red})
    savi = savi.rename('SAVI')
    return savi

# def NDBI(img):
#    ndbi = img.normalizedDifference(['B6', 'B5']).rename('NDBI')
#    return img.addBands(ndbi)#img.addBands(ndbi).updateMask(mask).clipToCollection(shp)


# Unions Two Images
# l5m
def unionimgsl5_7(a,b,month):
    n = len(new_bands)
    for i in range(0,n,1):
        # print(l8bands[i])
        imgA = a.select(l5bands[i])
        imgB = b.select(l7bands[i])
        if i == 0:
            mask = imgA.unmask(imgB)
        else:
            mask = mask.addBands(imgA.unmask(imgB))
    return ee.Image(mask.copyProperties(a, ["system:index","system:time_start"])).rename(new_bands)


def unionimgsl7_8(a,b,month):
    n = len(new_bands)
    for i in range(0,n,1):
        # print(l8bands[i])
        imgA = a.select(l7bands[i])
        imgB = b.select(l8bands[i])
        if i == 0:
            mask = imgA.unmask(imgB)
        else:
            mask = mask.addBands(imgA.unmask(imgB))
    return ee.Image(mask.copyProperties(a, ["system:index","system:time_start"])).rename(new_bands)

# l7m
def unionimgsl7_2(a,b,month):
    n = len(new_bands)
    for i in range(0,n,1):
        # print(l8bands[i])
        imgA = a.select(new_bands[i])
        imgB = b.select(s2bands[i])
        if i == 0:
            mask = imgA.unmask(imgB)
        else:
            mask = mask.addBands(imgA.unmask(imgB))
    return ee.Image(mask.copyProperties(a, ["system:index","system:time_start"])).rename(new_bands)

    # Unions Two Images
def unionimgs2(a,b,month):
    abands = list(a.bandNames().getInfo())# abands = geemap.image_band_names(a)# abands = list(a.getInfo().get('bands'))是错的
    bbands = list(b.bandNames().getInfo())# bbands = geemap.image_band_names(b)# bbands = list(b.getInfo().get('bands'))是错的
    # print(abands)
    # print(bbands)
    if len(abands)==len(bbands):
        n = len(abands)
        # print(abands)
        # print(bbands)
        for i in range(0,n,1):
            # print(l8bands[i])
            imgA = a.select(abands[i])
            imgB = b.select(bbands[i])
            if i == 0:
                mask = imgA.unmask(imgB)
            else:
                mask = mask.addBands(imgA.unmask(imgB))
    else:
        print("ERROR  XXXXXXXXXXXXXXXXXXXXXXXXX")
    return ee.Image(mask.copyProperties(a, ["system:index","system:time_start"])).select(abands).rename(new_bands)
    # return ee.Image(union.copyProperties(a, ["system:index","system:time_start"])).select(l8bands).rename(Nbands).unmask(-9999).clip(jx)

# 将image数据重新采样变为30m,使用第一个波段数据的投影信息
def fun(image):
    crs = image.select('NDVI').projection().getInfo()['crs']
    return image.resample('bilinear').reproject(**{'crs': crs, 'scale': 30.0})

def N_L(img):
    n_l = img.normalizedDifference(['NDVI', 'LSWI']).rename('N_L')
    return n_l

In [4]:
def handle(img,label,newB):
    # Overlay the points on the imagery to get training.
    training = img.select(newB).sampleRegions(**{
        'collection': sample,
        'properties': [label],
        'scale': 30
    })

    #  random uniforms to the training dataset.
    withRandom = training.randomColumn() #//样本点随机的排列

    # // 我们想保留一些数据进行测试，以避免模型过度拟合
    split = 0.8
    trainingPartition = withRandom.filter(ee.Filter.lt('random', split))#//筛选80%的样本作为训练样本
    testingPartition = withRandom.filter(ee.Filter.gte('random', split))#//筛选20%的样本作为测试样本

    # //分类方法选择smileCart() randomForest() minimumDistance libsvm
    classifier = ee.Classifier.smileRandomForest(200).train(trainingPartition,label,newB)

    classified = union.select([new_bands_]).classify(classifier)
    
    test = testingPartition.classify(classifier);              #//运用测试样本分类，确定要进行函数运算的数据集以及函数
    test_accuracy = test.errorMatrix(label, 'classification')
    print('总体精度',test_accuracy.accuracy().getInfo())
    
    return classified

In [6]:
#————————————————————————筛选单双季——————————————————————
#参数初始化
Climit = 25 #云量
startY = 1986
endY   = 2020
startM=3
endM=11
visParams = {min: 0.2, max: 0.8, 'palette': ['blue', 'white','green']};

a = []

# 按年筛选
# imglist = [] #存放每月mosaic后的影像
img = ee.Image.constant(0).clip(jx)
for y in range(startY,startY+1,1):
    # 按月筛选
    NDVI_imgs = []
    print(y,end='-')
    for m in range(startM,endM+1,1):        
        print(m,end='-')
        #更换2月天数
        if (y//4==0):
            days[2]=29
        else:
            days[2]=28
        start_date = str(y) + '-' + str(m).zfill(2) + '-01'
        end_date = str(y) + '-' + str(m).zfill(2) + '-' + str(days[m-1])
        

        if y >= 1984 and y < 2000:
            union = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
                .filterDate(start_date, end_date)\
                .sort('CLOUD_COVER', False)\
                .filterBounds(jx)\
                .map(maskL457sr)\
                .mosaic().select(l5bands).rename(new_bands)

        elif y >= 2000 and y <= 2012:
            l5m = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
                .filterDate(start_date, end_date)\
                .filterBounds(jx)\
                .sort('CLOUD_COVER', False)\
                .map(maskL457sr)\
                .mosaic().select(l5bands)

            l7m = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")\
                .filterDate(start_date, end_date)\
                .filterBounds(jx)\
                .sort('CLOUD_COVER', False)\
                .map(maskL457sr)\
                .mosaic().select(l7bands)

            union = unionimgsl5_7(l5m,l7m,1)

        elif y == 2013 or y == 2014:
            l7m = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")\
                .filterDate(start_date, end_date)\
                .filterBounds(jx)\
                .sort('CLOUD_COVER', False)\
                .map(maskL457sr)\
                .mosaic().select(l7bands)

            l8m = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                .filterDate(start_date, end_date)\
                .filterBounds(jx)\
                .sort('CLOUD_COVER', False)\
                .map(maskL8sr)\
                .mosaic().select(l8bands)

            union = unionimgsl7_8(l7m,l8m,1)

        elif y >= 2015 and y <= 2020:


            s2m = ee.ImageCollection('COPERNICUS/S2')\
                .filterDate(start_date, end_date)\
                .filterBounds(jx)\
                .sort('CLOUDY_PIXEL_PERCENTAGE', False)\
                .map(cloudfunction_ST2)\
                .mosaic().select(s2bands)#.clip(jx)

            l7m = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")\
                .filterDate(start_date, end_date)\
                .filterBounds(jx)\
                .sort('CLOUD_COVER', False)\
                .map(maskL457sr)\
                .mosaic().select(l7bands)

            l8m = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                .filterDate(start_date, end_date)\
                .filterBounds(jx)\
                .sort('CLOUD_COVER', False)\
                .map(maskL8sr)\
                .mosaic().select(l8bands)
            #.clip(jx)#.filterMetadata('CLOUD_COVER', 'less_than', Climit)

            union = unionimgsl7_8(l7m,l8m,1)
            union = unionimgsl7_2(union,s2m,1)
    
        union = union.addBands( NDVI(union))
        union = union.addBands( LSWI(union))
        
        NDVI_imgs.append(union.select(['NDVI','LSWI']))
        
    NDVI = ee.Image(NDVI_imgs)

    names = []
    for i in range(3,12):
        names.append('NDVI_{}'.format(i))
        names.append('LSWI_{}'.format(i))

    NDVI = NDVI.rename(names)
    temp = NDVI.sampleRegions(collection=sample, scale=500, geometries=True)
    
    double = temp.filter(ee.Filter.gte('NDVI_5',ee.Number(0.5)).And(ee.Filter.gte('NDVI_6',ee.Number(0.5)))\
                        .And(ee.Filter.greaterThan(**{'leftField': 'NDVI_6', 'rightField': 'NDVI_7'}))\
                        .And(ee.Filter.eq('Map',40)).Or(ee.Filter.neq('Map','40')))

    singal = temp.filter(ee.Filter.Not(ee.Filter.gte('NDVI_5',ee.Number(0.5)).And(ee.Filter.gte('NDVI_6',ee.Number(0.5)))\
                        .And(ee.Filter.greaterThan(**{'leftField': 'NDVI_6', 'rightField': 'NDVI_7'})))\
                        .And(ee.Filter.eq('Map',40)).Or(ee.Filter.neq('Map','40')))
    break
#     #----------------------------------数据处理--------------------------------------
    
#     for y in range(startY,startY+1,5):
    
#         start_date = str(y)+'-01-01'
#         end_date = str(y+1)+'-01-01'

#         print(str(y)+'_')

#         if y >= 1984 and y < 2000:


#             union = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
#                 .filterDate(start_date, end_date)\
#                 .filterBounds(jx)\
#                 .sort('CLOUD_COVER', False)\
#                 .map(maskL457sr)\
#                 .mosaic().select(l5bands).rename(new_bands)

#         elif y >= 2000 and y <= 2012:
#             l5m = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
#                 .filterDate(start_date, end_date)\
#                 .filterBounds(jx)\
#                 .sort('CLOUD_COVER', False)\
#                 .map(maskL457sr)\
#                 .mosaic().select(l5bands)

#             l7m = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")\
#                 .filterDate(start_date, end_date)\
#                 .filterBounds(jx)\
#                 .sort('CLOUD_COVER', False)\
#                 .map(maskL457sr)\
#                 .mosaic().select(l7bands)

#             union = unionimgsl5_7(l5m,l7m,1)

#         elif y == 2013 or y == 2014:
#             l7m = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")\
#                 .filterDate(start_date, end_date)\
#                 .filterBounds(jx)\
#                 .sort('CLOUD_COVER', False)\
#                 .map(maskL457sr)\
#                 .mosaic().select(l7bands)

#             l8m = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
#                 .filterDate(start_date, end_date)\
#                 .filterBounds(jx)\
#                 .sort('CLOUD_COVER', False)\
#                 .map(maskL8sr)\
#                 .mosaic().select(l8bands)

#             union = unionimgsl7_8(l7m,l8m,1)

#         elif y >= 2015 and y <= 2020:


#             s2m = ee.ImageCollection('COPERNICUS/S2')\
#                 .filterDate(start_date, end_date)\
#                 .filterBounds(jx)\
#                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 25))\
#                 .sort('CLOUDY_PIXEL_PERCENTAGE', False)\
#                 .map(cloudfunction_ST2)\
#                 .mosaic().select(s2bands)#.clip(jx)

#             l7m = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")\
#                 .filterDate(start_date, end_date)\
#                 .filterBounds(jx)\
#                 .sort('CLOUD_COVER', False)\
#                 .map(maskL457sr)\
#                 .mosaic().select(l7bands)

#             l8m = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
#                 .filterDate(start_date, end_date)\
#                 .filterBounds(jx)\
#                 .sort('CLOUD_COVER', False)\
#                 .map(maskL8sr)\
#                 .mosaic().select(l8bands)
#             #.clip(jx)#.filterMetadata('CLOUD_COVER', 'less_than', Climit)

#             union = unionimgsl7_8(l7m,l8m,1)
#             union = unionimgsl7_2(union,s2m,1)
        
#         union = union.addBands( NDVI(union))
#         union = union.addBands( LSWI(union))
#         union = union.addBands( N_L(union))

#         union = fun(union)
        
#         #————————————————————开始分类——————————————————————————————

#         training = union.select(new_bands_).sampleRegions(**{
#             'collection': double,
#             'properties': [label],
#             'scale': 30
#         })

#         #  random uniforms to the training dataset.
#         withRandom = training.randomColumn() #//样本点随机的排列

#         # // 我们想保留一些数据进行测试，以避免模型过度拟合
#         split = 0.8
#         trainingPartition = withRandom.filter(ee.Filter.lt('random', split))#//筛选80%的样本作为训练样本
#         testingPartition = withRandom.filter(ee.Filter.gte('random', split))#//筛选20%的样本作为测试样本

#         # //分类方法选择smileCart() randomForest() minimumDistance libsvm
#         classifier = ee.Classifier.smileRandomForest(200).train(trainingPartition,label,new_bands_)

#         classified = union.select(new_bands_).classify(classifier)

#         test = testingPartition.classify(classifier);              #//运用测试样本分类，确定要进行函数运算的数据集以及函数
#         test_accuracy = test.errorMatrix(label, 'classification')
#         print('总体精度',test_accuracy.accuracy().getInfo())


#         break

1986-3-4-5-6-7-8-9-10-11-

In [ ]:
# Map.addLayer(union,{},'union')
# Map.addLayer(classified.randomVisualizer(),{},'classified')
# Map

In [ ]:
# from GEE_xiong import ee_export
# file = r'C:\Users\xiong\Downloads\text\new1'

# ee_export(region=jx, image=union, outfile=file, scale=100)